 # Instructions:
 - For all cells marked with <font color="red">(CODE Needed)</font> below, replace `#CODE_HERE` with your solution


# 1. Load Spark Context (No CODE Needed)

In [12]:
from pyspark import SparkContext, SparkConf
myconf = SparkConf() \
        .setMaster("local[*]") 
        #.set("spark.driver.cores",4) \
        #.set("spark.executor.memory","2g") 
sc = SparkContext(conf=myconf)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-1-c24a73648561>:5 

# 2. Load some printing functions (No CODE Needed)

In [23]:
#These are some printing helper functions we will use to make the output more clear
from pprint import pprint
def title(s):
    pprint("---- %s -----" %s)    
    
def see(s, v):
    pprint("---- %s -----" %s)
    pprint(v)

# 3. Load each file in an RDD and add the year to the data (No CODE Needed)

In [24]:
def createOneYearRdd(year):
    file = "../data/flight-data/csv/%s"%year+"*"
    see("year:"+str(year), file)
    fileRDD =sc.textFile(file)
    linesRDD = fileRDD.filter(lambda x:  not x.startswith("DEST_")) \
              .filter(lambda line: line.count(',')==2)
    arrRDD = linesRDD.map(lambda line: (line.split(","), year))
    see("rdd count for year:"+str(year), arrRDD.count())
    title("rdd elements for year:"+str(year))
    for arr in (arrRDD.take(5)):
        print(arr)
    return arrRDD

separateRDDs  = list(map(createOneYearRdd, range(2010,2016)))
                 
see("arryay of RDDs", separateRDDs)

'---- year:2010 -----'
'../data/flight-data/csv/2010*'
'---- rdd count for year:2010 -----'
253
'---- rdd elements for year:2010 -----'
(['United States', 'Romania', '1'], 2010)
(['United States', 'Ireland', '264'], 2010)
(['United States', 'India', '69'], 2010)
(['Egypt', 'United States', '24'], 2010)
(['Equatorial Guinea', 'United States', '1'], 2010)
'---- year:2011 -----'
'../data/flight-data/csv/2011*'
'---- rdd count for year:2011 -----'
253
'---- rdd elements for year:2011 -----'
(['United States', 'Saint Martin', '2'], 2011)
(['United States', 'Guinea', '2'], 2011)
(['United States', 'Croatia', '1'], 2011)
(['United States', 'Romania', '3'], 2011)
(['United States', 'Ireland', '268'], 2011)
'---- year:2012 -----'
'../data/flight-data/csv/2012*'
'---- rdd count for year:2012 -----'
243
'---- rdd elements for year:2012 -----'
(['United States', 'Croatia', '1'], 2012)
(['United States', 'Ireland', '252'], 2012)
(['Egypt', 'United States', '13'], 2012)
(['United States', 'India', '

# 4. Merge all RDDs into one Rdd  (No CODE Needed)

In [25]:
allYearsRdd  =sc.union(separateRDDs)
see("allYearsRDD count", allYearsRdd.count())
see("allYearsRDD sample", allYearsRdd.takeSample(False, 10, 17))

'---- allYearsRDD count -----'
1488
'---- allYearsRDD sample -----'
[(['United States', 'Paraguay', '14'], 2014),
 (['Senegal', 'United States', '29'], 2011),
 (['United States', 'Samoa', '28'], 2010),
 (['United States', 'Thailand', '4'], 2015),
 (['United States', 'Afghanistan', '5'], 2012),
 (['United States', 'Belgium', '355'], 2010),
 (['United States', 'Sweden', '68'], 2011),
 (['United States', 'Sweden', '119'], 2015),
 (['Czech Republic', 'United States', '13'], 2015),
 (['Namibia', 'United States', '1'], 2011)]


# 5. Compute an RDD containing all flights which departed from  the US  <font color="red">(CODE Needed)</font>
- Hint: make 1 `filter` transformation

In [26]:
fromUSRdd = allYearsRdd.filter(lambda arr:arr[0][1]=='United States')

see("fromUSRdd count",fromUSRdd.count())

see("fromUSRdd take(10)",fromUSRdd.take(10))

'---- fromUSRdd count -----'
765
'---- fromUSRdd take(10) -----'
[(['Egypt', 'United States', '24'], 2010),
 (['Equatorial Guinea', 'United States', '1'], 2010),
 (['Costa Rica', 'United States', '477'], 2010),
 (['Senegal', 'United States', '29'], 2010),
 (['Guyana', 'United States', '17'], 2010),
 (['Malta', 'United States', '1'], 2010),
 (['Bolivia', 'United States', '46'], 2010),
 (['Anguilla', 'United States', '21'], 2010),
 (['Turks and Caicos Islands', 'United States', '136'], 2010),
 (['Saint Vincent and the Grenadines', 'United States', '1'], 2010)]


### Expectated output for step 5.

<pre>
'---- fromUSRdd count -----'
765
'---- fromUSRdd take(10) -----'
[(['Egypt', 'United States', '24'], 2010),
 (['Equatorial Guinea', 'United States', '1'], 2010),
 (['Costa Rica', 'United States', '477'], 2010),
 (['Senegal', 'United States', '29'], 2010),
 (['Guyana', 'United States', '17'], 2010),
 (['Malta', 'United States', '1'], 2010),
 (['Bolivia', 'United States', '46'], 2010),
 (['Anguilla', 'United States', '21'], 2010),
 (['Turks and Caicos Islands', 'United States', '136'], 2010),
 (['Saint Vincent and the Grenadines', 'United States', '1'], 2010)]
</pre>

# 6. Create an RDD with a composite key (destination, year) and a value n which is the corresponding number of flights  as an integer <font color="red">(CODE Needed)</font>
- Use the `int()` function to convert a string to an integer
- Hint: make 1 `map` transformation

In [27]:
compositeKeyRdd = fromUSRdd.map(lambda arr:((arr[0][0],arr[1]),int(arr[0][2])))

see("compositeKeyRdd count",compositeKeyRdd.count())

see("compositeKeyRdd take(10)",compositeKeyRdd.take(10))

'---- compositeKeyRdd count -----'
765
'---- compositeKeyRdd take(10) -----'
[(('Egypt', 2010), 24),
 (('Equatorial Guinea', 2010), 1),
 (('Costa Rica', 2010), 477),
 (('Senegal', 2010), 29),
 (('Guyana', 2010), 17),
 (('Malta', 2010), 1),
 (('Bolivia', 2010), 46),
 (('Anguilla', 2010), 21),
 (('Turks and Caicos Islands', 2010), 136),
 (('Saint Vincent and the Grenadines', 2010), 1)]


### Exepected output for step 6
<pre>
'---- compositeKeyRdd count -----'
765
'---- compositeKeyRdd take(10) -----'
[(('Egypt', 2010), 24),
 (('Equatorial Guinea', 2010), 1),
 (('Costa Rica', 2010), 477),
 (('Senegal', 2010), 29),
 (('Guyana', 2010), 17),
 (('Malta', 2010), 1),
 (('Bolivia', 2010), 46),
 (('Anguilla', 2010), 21),
 (('Turks and Caicos Islands', 2010), 136),
 (('Saint Vincent and the Grenadines', 2010), 1)]
 </pre>

# 7. Sum all the flights for the same destination and the same year <font color="red">(CODE Needed)</font>
- Hint: make 1 `reduceByKey` transformation

In [28]:
groupedRdd = compositeKeyRdd.reduceByKey(lambda x,y:x+y)

see("groupedRdd count",groupedRdd.count())

see("groupedRdd take(10)",groupedRdd.take(10))

'---- groupedRdd count -----'
765
'---- groupedRdd take(10) -----'
[(('Suriname', 2010), 12),
 (('Thailand', 2010), 16),
 (('Ethiopia', 2010), 12),
 (('Barbados', 2010), 130),
 (('Cyprus', 2010), 2),
 (('Fiji', 2010), 53),
 (('Federated States of Micronesia', 2010), 46),
 (('French Guiana', 2010), 4),
 (('Netherlands', 2010), 586),
 (('China', 2010), 448)]


### Exepected output for step  7 
<pre>
'---- groupedRdd count -----'
765
'---- groupedRdd take(10) -----'
[(('Ethiopia', 2010), 12),
 (('Fiji', 2010), 53),
 (('Federated States of Micronesia', 2010), 46),
 (('French Guiana', 2010), 4),
 (('Netherlands', 2010), 586),
 (('Denmark', 2010), 98),
 (('Belgium', 2010), 408),
 (('Guatemala', 2010), 386),
 (('Vietnam', 2010), 1),
 (('South Korea', 2010), 683)]
 </pre>

# 8. Sort the summed data by the highest sum <font color="red">(CODE Needed)</font>
- Hint: make 1 `map` transformation, 1 `sortByKey(ascending=False)`, then 1 `map`


In [29]:
sortedByCountRdd = groupedRdd.map(lambda arr:(arr[1],arr[0])).sortByKey(ascending=False).map(lambda arr:(arr[1],arr[0]))

see("sortedByCountRdd count",sortedByCountRdd.count())

see("sortedByCountRdd take(10)",sortedByCountRdd.take(10))


'---- sortedByCountRdd count -----'
765
'---- sortedByCountRdd take(10) -----'
[(('United States', 2015), 370002),
 (('United States', 2014), 358354),
 (('United States', 2011), 352742),
 (('United States', 2010), 348113),
 (('United States', 2012), 347452),
 (('United States', 2013), 343132),
 (('Canada', 2011), 8514),
 (('Canada', 2015), 8399),
 (('Canada', 2010), 8271),
 (('Canada', 2012), 8034)]


### Exepected output for step  8
<pre>
'---- sortedByCountRdd count -----'
765
'---- sortedByCountRdd take(10) -----'
[(('United States', 2015), 370002),
 (('United States', 2014), 358354),
 (('United States', 2011), 352742),
 (('United States', 2010), 348113),
 (('United States', 2012), 347452),
 (('United States', 2013), 343132),
 (('Canada', 2011), 8514),
 (('Canada', 2015), 8399),
 (('Canada', 2010), 8271),
 (('Canada', 2012), 8034)]
 </pre>

# 9. Compute the Sum of all Trips <font color="red">(CODE Needed)</font>
- Hint: make 1 `map` , 1 `sum`


In [30]:
sumAllTrips = sortedByCountRdd.map(lambda arr: int(arr[1])).reduce(lambda x,y: x+y)
see("sumAllTrips", sumAllTrips)

'---- sumAllTrips -----'
2352144


### Exepected output for step  9
<pre>
'---- sumAllTrips -----'
2352144
 </pre>

# 10. Compute the Sum of Each Year's Trip <font color="red">(CODE Needed)</font>
- Hint: make 1 `map` , 1 `reduceByKey`, 1 `collectAsMap`


In [62]:
TPY = groupedRdd.map(lambda arr:(arr[0][1],int(arr[1]))).reduceByKey(lambda x,y:x+y).collectAsMap()

see("TPY", TPY)


'---- TPY -----'
{2010: 385434,
 2011: 390613,
 2012: 385244,
 2013: 380985,
 2014: 397960,
 2015: 411908}


### Exepected output for step  10
<pre>
'---- TPY -----'
{2010: 385434,
 2011: 390613,
 2012: 385244,
 2013: 380985,
 2014: 397960,
 2015: 411908}
 </pre>

# 11. Use the above sums to compute percent Per Year, percent from Total for each country, year pair <font color="red">(CODE Needed)</font>
- Hint: make 1 `map`
- use the function `percent`
- use TPY as a lookup table


In [63]:
def percent(number, total):
    return 100.0* number/float(total)

stats = sortedByCountRdd.map(lambda arr:(arr,percent(arr[1],TPY[arr[0][1]]),percent(arr[1],sumAllTrips)))
                                                    

see("stats count",stats.count())

see("stats take(10)",stats.take(10))

'---- stats count -----'
765
'---- stats take(10) -----'
[((('United States', 2015), 370002), 89.8263689950183, 15.730414464420546),
 ((('United States', 2014), 358354), 90.04774349180822, 15.235206688025903),
 ((('United States', 2011), 352742), 90.30472616118767, 14.99661585345115),
 ((('United States', 2010), 348113), 90.31714898011073, 14.799816677890469),
 ((('United States', 2012), 347452), 90.19011327885704, 14.771714656925766),
 ((('United States', 2013), 343132), 90.06443823247635, 14.588052432164018),
 ((('Canada', 2011), 8514), 2.1796509588774566, 0.36196763463461423),
 ((('Canada', 2015), 8399), 2.039047554308244, 0.3570784781884102),
 ((('Canada', 2010), 8271), 2.145892682015598, 0.3516366344917658),
 ((('Canada', 2012), 8034), 2.0854315706409445, 0.3415607207721976)]


### Exepected output for step  11
<pre>
'---- stats count -----'
765
'---- stats take(10) -----'
[(('United States', 2015), 370002, 89.8263689950183, 15.730414464420546),
 (('United States', 2014), 358354, 90.04774349180822, 15.235206688025903),
 (('United States', 2011), 352742, 90.30472616118767, 14.99661585345115),
 (('United States', 2010), 348113, 90.31714898011073, 14.799816677890469),
 (('United States', 2012), 347452, 90.19011327885704, 14.771714656925766),
 (('United States', 2013), 343132, 90.06443823247635, 14.588052432164018),
 (('Canada', 2011), 8514, 2.1796509588774566, 0.36196763463461423),
 (('Canada', 2015), 8399, 2.039047554308244, 0.3570784781884102),
 (('Canada', 2010), 8271, 2.145892682015598, 0.3516366344917658),
 (('Canada', 2012), 8034, 2.0854315706409445, 0.3415607207721976)]
 </pre>

# 12. Transform the above table into the format below <font color="red">(CODE Needed)</font>
- Hint: make 1 `map`, 1 `groupByKey`, 1 `mapValues` 
- you will also need the python functions `list`, `sorted`, `dict`
- use TPY as a lookup table


In [119]:
preTable = stats.map(lambda arr: (arr[0][0][0],(arr[0][0][1],(arr[1],arr[2])))).groupByKey().mapValues(lambda lst:dict(lst)).sortByKey()

see("preTable count",preTable.count())

see("preTable take(5)",preTable.take(5))

'---- preTable count -----'
165
'---- preTable take(5) -----'
[('Afghanistan',
  {2010: (0.002853925704530477, 0.0004676584426803801),
   2011: (0.0020480629164927946, 0.0003401152310402764),
   2012: (0.0012978787469759426, 0.00021257201940017278)}),
 ('Algeria',
  {2013: (0.0005249550507237817, 8.50288077600691e-05),
   2014: (0.002261533822494723, 0.000382629634920311),
   2015: (0.0009710906318886742, 0.0001700576155201382)}),
 ('Angola',
  {2010: (0.0036322690784933347, 0.0005952016543204838),
   2011: (0.0033281022393007913, 0.0005526872504404493),
   2012: (0.003114908992742262, 0.0005101728465604147),
   2013: (0.0031497303043426907, 0.0005101728465604147),
   2014: (0.003266659965825711, 0.0005526872504404493),
   2015: (0.003641589869582528, 0.0006377160582005184)}),
 ('Anguilla',
  {2010: (0.0054484036177400025, 0.0008928024814807257),
   2011: (0.005376165155793586, 0.0008928024814807257),
   2012: (0.004931939238508582, 0.0008077736737206565),
   2013: (0.0057745055579616,

### Exepected output for step  12
<pre>
'---- preTable count -----'
165
'---- preTable take(10) -----'
[('Afghanistan',
  {2010: (11, 0.002853925704530477, 0.0004676584426803801),
   2011: (8, 0.0020480629164927946, 0.0003401152310402764),
   2012: (5, 0.0012978787469759426, 0.00021257201940017278)}),
 ('Algeria',
  {2013: (2, 0.0005249550507237817, 8.50288077600691e-05),
   2014: (9, 0.002261533822494723, 0.000382629634920311),
   2015: (4, 0.0009710906318886742, 0.0001700576155201382)}),
 ('Angola',
  {2010: (14, 0.0036322690784933347, 0.0005952016543204838),
   2011: (13, 0.0033281022393007913, 0.0005526872504404493),
   2012: (12, 0.003114908992742262, 0.0005101728465604147),
   2013: (12, 0.0031497303043426907, 0.0005101728465604147),
   2014: (13, 0.003266659965825711, 0.0005526872504404493),
   2015: (15, 0.003641589869582528, 0.0006377160582005184)}),
 ('Anguilla',
  {2010: (21, 0.0054484036177400025, 0.0008928024814807257),
   2011: (21, 0.005376165155793586, 0.0008928024814807257),
   2012: (19, 0.004931939238508582, 0.0008077736737206565),
   2013: (22, 0.0057745055579616, 0.0009353168853607602),
   2014: (34, 0.008543572218313398, 0.0014454897319211748),
   2015: (41, 0.00995367897685891, 0.0017430905590814169)}),
 ('Antigua and Barbuda',
  {2010: (123, 0.03191207833247715, 0.00522927167724425),
   2011: (146, 0.0373771482259935, 0.006207102966485045),
   2012: (145, 0.03763848366230233, 0.006164588562605011),
   2013: (123, 0.03228473561951258, 0.00522927167724425),
   2014: (115, 0.028897376620765906, 0.0048891564462039735),
   2015: (126, 0.030589354904493236, 0.005356814888884354)})]
 </pre>

# 13. Transform the above table into the format below <font color="red">(CODE Needed)</font>
- Hint: make 1 `map`, you will need more than one line in the map, so create a mapping function

In [ ]:
Table = #CODE_HERE

see("Table count",Table.count())

see("Table take(3)",Table.take(3))

### Exepected output for step  13
<pre>
'---- Table count -----'
165
'---- Table take(3) -----'
[{'Country': 'Afghanistan',
  'PcntTotal-2010': 0.0004676584426803801,
  'PcntTotal-2011': 0.0003401152310402764,
  'PcntTotal-2012': 0.00021257201940017278,
  'PcntYear-2010': 0.002853925704530477,
  'PcntYear-2011': 0.0020480629164927946,
  'PcntYear-2012': 0.0012978787469759426,
  'Tot-2010': 11,
  'Tot-2011': 8,
  'Tot-2012': 5},
 {'Country': 'Algeria',
  'PcntTotal-2013': 8.50288077600691e-05,
  'PcntTotal-2014': 0.000382629634920311,
  'PcntTotal-2015': 0.0001700576155201382,
  'PcntYear-2013': 0.0005249550507237817,
  'PcntYear-2014': 0.002261533822494723,
  'PcntYear-2015': 0.0009710906318886742,
  'Tot-2013': 2,
  'Tot-2014': 9,
  'Tot-2015': 4},
 {'Country': 'Angola',
  'PcntTotal-2010': 0.0005952016543204838,
  'PcntTotal-2011': 0.0005526872504404493,
  'PcntTotal-2012': 0.0005101728465604147,
  'PcntTotal-2013': 0.0005101728465604147,
  'PcntTotal-2014': 0.0005526872504404493,
  'PcntTotal-2015': 0.0006377160582005184,
  'PcntYear-2010': 0.0036322690784933347,
  'PcntYear-2011': 0.0033281022393007913,
  'PcntYear-2012': 0.003114908992742262,
  'PcntYear-2013': 0.0031497303043426907,
  'PcntYear-2014': 0.003266659965825711,
  'PcntYear-2015': 0.003641589869582528,
  'Tot-2010': 14,
  'Tot-2011': 13,
  'Tot-2012': 12,
  'Tot-2013': 12,
  'Tot-2014': 13,
  'Tot-2015': 15}]
 </pre>

# 14. Optional: Convert the data to a Pandas Dataframe <font color="green">(No CODE Needed)</font>


In [ ]:
import pandas as pd

pd.DataFrame(data=Table.collect())